In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd

base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')
base=base.drop('dateCrawled', axis =1)
base=base.drop('dateCreated', axis =1)
base=base.drop('nrOfPictures', axis =1)
base=base.drop('postalCode', axis =1)
base=base.drop('lastSeen', axis =1)



In [10]:
base['name'].value_counts()
base=base.drop('name', axis =1)
base['seller'].value_counts()
base=base.drop('seller', axis =1)
base['offerType'].value_counts()
base=base.drop('offerType', axis =1)

In [11]:
i1 = base.loc[base.price <= 10]
base = base[base.price>10]
i2 = base.loc[base.price>350000]
base = base.loc[base.price < 350000]


In [14]:

base['vehicleType'].value_counts()
base['gearbox'].value_counts()
base['model'].value_counts()
base['fuelType'].value_counts()
base['notRepairedDamage'].value_counts()
val = { 'vehicleType' : 'limousine', 'gearbox' : 'manuell', 'model' : 'golf', 'fuelType' : 'benzin', 'notRepairedDamage' : 'nein'}
base = base.fillna(value = val)
base

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,nein
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,limousine,2005,manuell,0,golf,20000,1,benzin,sonstige_autos,nein
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,nein


In [27]:
X = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [19]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
    ("c0", OneHotEncoder(), [0]),
    ("c1", OneHotEncoder(), [1]),
    ("c3", OneHotEncoder(), [3]),
    ("c5", OneHotEncoder(), [5]),
    ("c8", OneHotEncoder(), [8]),
    ("c9", OneHotEncoder(), [9]),
    ("c10", OneHotEncoder(), [10]),
    
    ], remainder='passthrough')



In [24]:
X = ct.fit_transform(X).toarray() #tentei rodar 5 vezes mas meu ambiente crashava e inutilizava o meu computador :(

In [28]:
X[: , 3]

array(['manuell', 'manuell', 'automatik', ..., 'manuell', 'manuell',
       'manuell'], dtype=object)

In [29]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasRegressor

In [33]:
def criaRede():
    regressor = Sequential()
    regressor.add(Dense(units=158, activation='relu', input_dim = 316))
    regressor.add(Dense(units=158, activation='relu'))
    regressor.add(Dense(units=1, activation='linear'))

    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    regressor.fit(X, real_price, batch_size=300, epochs=100)
    return regressor

In [34]:
regressor = KerasRegressor(build_fn=criaRede, epochs=100, batch_size=300)
results = cross_val_score(estimator=regressor, X=X, y=preco_real, cv =10, scoring='neg_mean_absolute_error')
media = results.mean()
desvio = results.std()

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\shink\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\shink\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "C:\Users\shink\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scikeras\wrappers.py", line 925,